In [1]:
# importing required pakcages
import pandas as pd
from tqdm import tqdm

In [2]:
# config
data_config = 'MovieLens1M'
config = {
    'MovieLens1M':
    {
        'ds_name': 'MovieLens1M',
        'ds_fullname': 'MovieLens1M_data_map',
        'ds_catname': 'MovieLens1M_genre_map',
        'ds_fmt': 'txt',
        # 'ds_acronym': 'BX',
        'ds_rate': 10,
        'ds_toprate': 200,
        'ds_sep': ',',
        # 'ds_encode': 'ISO-8859–1'
    }
}

In [3]:
# loading the Book-Crossing dataset
dataset = pd.read_csv(f"datasets/{data_config}/raw/{data_config}_data_map.{config[data_config]['ds_fmt']}", sep=f"{config[data_config]['ds_sep']}")
dataset.columns = ['UserID', 'ItemID', 'Rating']

In [4]:
dataset.head()

,UserID,ItemID,Rating
0,0,1,3
1,0,2,3
2,0,3,4
3,0,4,5
4,0,5,3


In [5]:
# what columns or attributes the dataset is included?
dataset.columns

Index(['UserID', 'ItemID', 'Rating'], dtype='object')

In [6]:
# statistics on explicit dataset
print("Original dataset statistics: ")
print(f"> No. of users: {len(dataset['UserID'].unique())}")
print(f"> No. of Items: {len(dataset['ItemID'].unique())}")
print(f"> No. of Interaction: {dataset.shape[0]}")

Original dataset statistics: 
> No. of users: 6040
> No. of Items: 3706
> No. of Interaction: 1000208


In [7]:
# checking the number of times in which a rating happened - (0: implicit, 1-10: explicit)
dataset['Rating'].value_counts()

4    348971
3    261197
5    226309
2    107557
1     56174
Name: Rating, dtype: int64

In [8]:
# This method return a dataframe in which the specified values are removed from a specific column. We can use it to generate the implicit/explicit dataset.
# To do this, we can remove the 0 or 1-10 values from the `Book-Rating` column.
def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

In [9]:
# To create the explicit dataset, we first remove those rows which includes 0 on Book-Rating columns (implicit ones), then we save the new generated daaset.
if data_config == 'ClothingFit':
    dataset = filter_rows_by_values(dataset, "Rating", [0])

    print(dataset['Rating'].value_counts())

    # statistics on explicit dataset
    print("Explicit dataset statistics: ")
    print(f"> No. of users: {len(dataset['UserID'].unique())}")
    print(f"> No. of Books: {len(dataset['ItemID'].unique())}")
    print(f"> No. of Interaction: {dataset.shape[0]}")

### Remove user with more than TopRate interactions

In [10]:
# To check if there is any user with more than toprate interaction in the preprocessed dataset
# The correct output will be zero
uid_value_counts = dataset['UserID'].value_counts()
print(f"The numebr of users with more than {config[data_config]['ds_toprate']} interactions: {uid_value_counts[uid_value_counts > config[data_config]['ds_toprate']].count()}")

The numebr of users with more than 200 interactions: 1578


In [11]:
# To remove the users with fewer than 5 interaction we first count the number of interactino per user and add a new column (`Count`) in the dataframe.
# This column shows the number of interaction per user in the dataset
users_counts = dataset['UserID'].value_counts()
users_counts = users_counts.to_dict() #converts to dictionary
dataset['Count'] = dataset['UserID'].map(users_counts)

In [12]:
# remove between toprate to max value
# dataset = filter_rows_by_values(dataset, "Count", list(range(config[data_config]['ds_toprate'], max(dataset['Count']) + 1)))

In [13]:
# statistics on explicit dataset after removing users with more than 200 int.
print(f"New dataset statistics (users with interactions < {config[data_config]['ds_toprate']}): ")
print(f"> No. of users: {len(dataset['UserID'].unique())}")
print(f"> No. of Books: {len(dataset['ItemID'].unique())}")
print(f"> No. of Interaction: {dataset.shape[0]}")

New dataset statistics (users with interactions < 200): 
> No. of users: 6040
> No. of Books: 3706
> No. of Interaction: 1000208


### Iteratively remove users and items with fewer than `ds_rate` interactions

In [14]:
list(range(config[data_config]['ds_rate']))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [15]:
user_interaction, item_interaction = 1, 1

while user_interaction != 0 or item_interaction != 0:
    print(f"The current number of user and item with < {config[data_config]['ds_rate']} interactions: ")
    # user side fewer than ds_rate cheking
    uid_value_counts = dataset['UserID'].value_counts()
    user_interaction = uid_value_counts[uid_value_counts < config[data_config]['ds_rate']].count()
    print(f"No. of users < {config[data_config]['ds_rate']} ineractions: {user_interaction}")

    users_counts = dataset['UserID'].value_counts()
    users_counts = users_counts.to_dict() #converts to dictionary
    dataset['Count'] = dataset['UserID'].map(users_counts)

    dataset = filter_rows_by_values(dataset, "Count", list(range(config[data_config]['ds_rate'])))

    # item side fewer than ds_rate cheking
    bid_value_counts = dataset['ItemID'].value_counts()
    item_interaction = bid_value_counts[bid_value_counts < config[data_config]['ds_rate']].count()
    print(f"No. of items < {config[data_config]['ds_rate']} ineractions: {item_interaction}")

    items_counts = dataset['ItemID'].value_counts()
    items_counts = items_counts.to_dict() #converts to dictionary
    dataset['Count'] = dataset['ItemID'].map(items_counts)

    dataset = filter_rows_by_values(dataset, "Count", list(range(config[data_config]['ds_rate'])))

The current number of user and item with < 10 interactions: 
No. of users < 10 ineractions: 0
No. of items < 10 ineractions: 446
The current number of user and item with < 10 interactions: 
No. of users < 10 ineractions: 0
No. of items < 10 ineractions: 0


In [16]:
# statistics on 5 rate explicit dataset (after pre-processing)
print(f"No. of Users: {len(dataset['UserID'].unique())}")
print(f"No. of Items: {len(dataset['ItemID'].unique())}")
print(f"No. of Interaction: {dataset.shape[0]}")

No. of Users: 6040
No. of Items: 3260
No. of Interaction: 998538


In [17]:
# Before we save the preprocessed explicit dataset (5Rate) we first remove the added column which is `Count`
del dataset['Count']

### User and Item ID Mapping

In [18]:
uid_to_index = dict()
iid_to_index = dict()

last_user_id = 0
last_item_id = 0

for eachline in tqdm(dataset.iterrows()):
  # add a new user id with an index
  if eachline[1][0] not in uid_to_index.keys():
    uid_to_index[eachline[1][0]] = last_user_id
    last_user_id += 1
  # add a new book id with an index
  if eachline[1][1] not in iid_to_index.keys():
    iid_to_index[eachline[1][1]] = last_item_id
    last_item_id += 1

998538it [01:42, 9778.78it/s] 


In [19]:
# write the file with new mapped indices into a txt file
# mapped_dataset = open(f"dataset/{data_config}/{config[data_config]['ds_acronym']}-Book-Explicit-{config[data_config]['ds_rate']}Rate-Map.csv", 'w')
mapped_dataset = open(f"datasets/{data_config}/{data_config}_data.txt", 'w')

for eachline in tqdm(dataset.iterrows()):
    mapped_dataset.write(str(uid_to_index[eachline[1][0]]) + "," + str(iid_to_index[eachline[1][1]]) + "," + str(eachline[1][2]) + "\n")
mapped_dataset.close()

998538it [02:05, 7972.30it/s]


In [20]:
categories = pd.read_csv(f"datasets/{data_config}/raw/{data_config}_cat_map.{config[data_config]['ds_fmt']}", sep=f"{config[data_config]['ds_sep']}")
categories.columns = ['ItemID', 'Categories']

In [21]:
categories.head()

,ItemID,Categories
0,573,Adventure|Children's|Fantasy
1,1333,Comedy|Romance
2,450,Comedy|Drama
3,1334,Comedy
4,255,Action|Crime|Thriller


In [22]:
mapped_genre = open(f"datasets/{data_config}/{data_config}_cat.txt", 'w')

for eachline in tqdm(categories.itertuples(index=True)):
    if eachline.ItemID in iid_to_index.keys():
        mapped_genre.write(str(iid_to_index[eachline.ItemID]) + "," + str(eachline.Categories) + "\n")

3705it [00:00, 292680.97it/s]
